# Study batery influence on residual value



In [ ]:
from core.gsheet_utils import load_excel_data, get_google_client
from core.s3.s3_utils import S3Service
from core.sql_utils import get_sqlalchemy_engine
from core.spark_utils import *
import pandas as pd
import numpy as np 
import plotly.express as px

In [ ]:
df_scrapping = load_excel_data("Courbes de tendance", "Courbes OS")
df_scrapping = pd.DataFrame(columns=df_scrapping[:1][0], data=df_scrapping[1:])

In [ ]:
df_scrapping['Odomètre (km)'] = df_scrapping['Odomètre (km)'].apply(lambda x: float(x.replace(',', '')))
df_scrapping['SoH'] = df_scrapping['SoH'].apply(lambda x: float(x.replace('%', '')))
df_scrapping['price'] = df_scrapping['price'].replace('', np.nan).astype(float)

In [ ]:
engine = get_sqlalchemy_engine()
df_dbeaver = pd.read_sql("""SELECT o.oem_name, m.make_name, vm.model_name, vm.type, vm.version, vm.autonomy, b.battery_chemistry, b.capacity, b.net_capacity FROM vehicle_model vm
join battery b on b.id=vm.battery_id 
join make m on m.id=vm.make_id
join oem o on o.id=vm.oem_id""", engine)

In [ ]:
df = df_scrapping.merge(df_dbeaver, right_on=["model_name", "type"], left_on=['Modèle', 'Type'])[['oem_name', 'make_name', 'model_name','type','version','autonomy','battery_chemistry','capacity','net_capacity', 'SoH', 'Année', 'price', 'Odomètre (km)']]
df.rename(columns={'Odomètre (km)': 'odometer'}, inplace=True)

## Battery influence on VR

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Matrice de corrélation
corr_vars = ['price', 'autonomy', 'odometer', 'SoH', 'net_capacity', 'Année']
corr_matrix = df[df['price'] > 0][corr_vars].corr()


px.imshow(corr_matrix[['price', 'odometer']].round(2), text_auto=True)


Le SoH et l'age sont corrélés au prix mais ils sont aussi très corrélés a l'odometer.

Deux approches sont possible:
- Prendre en compte la capacité d el batterie et l'odomètre pour expliquer le prix sans reduire la corrélation SoH et Année.
- Traiter la corrélation du SoH et / ou année. 

### Retirer l'impact du SoH et de l'année sur le prix

**Année comme variable continue**

In [ ]:
import statsmodels.formula.api as smf
df_analysis = df[df['price'] > 0].dropna(subset=['price', 'autonomy', 'odometer', 'SoH', 'Année', 'net_capacity'])
df_analysis['Année'] = df_analysis['Année'].astype(int)
model = smf.ols('price ~ autonomy + net_capacity + odometer + SoH + Année', data=df_analysis).fit()

print("\n📊 Interprétation:")
print(f"- Chaque km d'autonomie supplémentaire augmente le prix de {model.params['autonomy']:.2f}€")
print(f"- Chaque kWh de capacité nette supplémentaire augmente le prix de {model.params['net_capacity']:.2f}€")
print(f"- Chaque 1000 km parcourus diminue le prix de {-model.params['odometer'] * 1000:.2f}€")
print(f"- Chaque point de SoH supplémentaire augmente le prix de {model.params['SoH']:.2f}€")
print(f"- Chaque année supplémentaire diminue le prix de {model.params['Année']:.2f}€")
print(f"- R² = {model.rsquared:.3f} → {model.rsquared*100:.1f}% de la variance expliquée")




In [ ]:
soh_mean = df_analysis['SoH'].mean()
year_mean = df_analysis['Année'].mean()
model = smf.ols('price ~ SoH + Année', data=df_analysis).fit()
df_analysis['price_adjusted_continue'] = df_analysis['price'] - (
    model.params['SoH'] * (df_analysis['SoH'] - soh_mean)
    + model.params['Année'] * (df_analysis['Année'] - year_mean)
)


In [ ]:
corr_matrix = df_analysis[df_analysis['price'] > 0][['price_adjusted_continue', 'SoH', 'Année', 'odometer', 'autonomy', 'net_capacity']].corr()
px.imshow(corr_matrix[['price_adjusted_continue']].round(2), text_auto=True)

Pour le cas continue les facteurs de corrélations pour l'année et le SoH on diminuent mais comme on pouvat le prévoir celui de l'odomètre aussi.

In [ ]:
# Créer des catégories d'autonomie
df_cat_adj = df_analysis.copy()
df_cat_adj['autonomy_category'] = pd.cut(
    df_cat_adj['autonomy'], 
    bins=[0, 300, 400, 500, 1000],
    labels=['<300 km', '300-400 km', '400-500 km', '>500 km']
)

# Définir l'ordre des catégories
category_order = ['<300 km', '300-400 km', '400-500 km', '>500 km']

# Box plot : Prix par catégorie d'autonomie
fig = px.box(
    df_cat_adj,
    x="autonomy_category",
    y="price",
    color="autonomy_category",
    category_orders={"autonomy_category": category_order},
    title="Distribution des prix par catégorie d'autonomie"
)
fig.show()
fig.write_html("distrib_price_adj_by_autonomy_category.html")



# Scatter : Odomètre vs Prix par catégorie
fig = px.scatter(
    df_cat_adj,
    x="odometer",
    y="price",
    color="autonomy_category",
    category_orders={"autonomy_category": category_order},
    trendline="ols",
    title="Dépréciation selon l'odomètre, par catégorie d'autonomie"
)
fig.show()
fig.write_html("scatter_price_adj_odometer.html")

# Visualisation : Prix observé vs net_capcité
fig = px.scatter(
    df_analysis,
    x="net_capacity",
    y="price",
    hover_data=["model_name", "type", "SoH", "price"],
    title=f"Prix ajusté en fonction de la capacité nette",
    trendline="ols"
)
fig.show()
fig.write_html("scatter_price_adj_net_capacity.html")


# Visualisation : Prix observé vs autonomie
fig = px.scatter(
    df_analysis,
    x="autonomy",
    y="price",
    hover_data=["model_name", "type", "SoH", "price"],
    title=f"Prix ajusté en fonction de l'autonomie",
    trendline="ols"
)
fig.show()
fig.write_html("scatter_price_adj_autonomie.html")



In [ ]:
X = df_analysis[['autonomy', 'odometer']]
y = df_analysis['price']

X = sm.add_constant(X)
model = sm.OLS(y, X).fit()
print("\n📊 Résultat de la régression:")
print(f"- Chaque km d'autonomie supplémentaire augmente le prix de {model.params['autonomy']:.2f}€")
print(f"- Chaque 1000 km parcourus diminue le prix de {-model.params['odometer'] * 1000:.2f}€")
# print(f"- Chaque kWh de capacité nette augmente le prix de {model.params['net_capacity']:.2f}€")
print(f"- R² = {model.rsquared:.3f} → {model.rsquared*100:.1f}% de la variance expliquée")